In [1]:
from math import factorial
import matplotlib.pyplot as plt
import numpy as np

def offeredTraffic(calls_per_hour, hours_per_call): # A0
    return calls_per_hour*hours_per_call

def erlangb(n, A0):
    denom = 0
    for i in range(n):
        denom += (A0**i)/(factorial(i))
    E1 = (A0**n)/(factorial(n))/denom
    return E1

In [2]:
def trafficSimulation(n, numCalls, meanCallDuration):

    callDurations = np.random.exponential(scale=meanCallDuration, size=numCalls)
    # print("Mean call duration =",callDurations.mean())
    # plt.hist(callDurations, bins=numCalls//10)
    # plt.title("Call duration frequency")
    # plt.ylabel=("frequency")
    # plt.show()

    callStarts = np.random.uniform(size=numCalls)
    # plt.hist(callStarts, bins=numCalls//10)
    # plt.show()
    callStarts.sort()

    # Calls = Numpy array of [[call0_start call0_end]; [call1_start call1_end];...
    calls = np.stack((callStarts, np.add(callStarts,callDurations)),axis=1)

    # plt.hist(load, bins=1000)
    # plt.plot(calls)
    # plt.show()
    
    channels = np.zeros(n)
    
    time = 0
    callsSucceeded = 0
    for i, call in enumerate(calls):
        time = call[0]
        channelFound = False
        for j in range(n):
            if not(channelFound):
                if channels[j] < time:
                    # print(j,"used")
                    # print("Channel {} free. Adding call time {}".format(j, call[1]))
                    channels[j] = call[1]
                    channelFound = True
        if channelFound:
            callsSucceeded += 1
        # else:
        #     print("Call {} failed.".format(i))
    # print(time, call, channels)
    callsFailed = numCalls - callsSucceeded
    # print("{}% of calls failed ({} succeeded, {} failed).".format(100*callsFailed/numCalls, callsSucceeded, callsFailed))
    return callsFailed/numCalls, callsFailed


In [3]:
meanCallDuration = 3/60 # 3 mins in hrs
n = 56
numCalls = 1000
repetitions = 100
# steps:
print("Erlang B = {}".format(100*erlangb(n, offeredTraffic(numCalls, meanCallDuration))))
fail_rate, fails = 0,0
for i in range(repetitions):
    new_acc, new_fails = trafficSimulation(n, numCalls, meanCallDuration)
    fail_rate += new_acc
    fails += new_fails

print("Call fail rate in 100 simulations = {}% (mean of {} fails in {} calls)".format(100*fail_rate/repetitions, fails/100, numCalls))



Erlang B = 4.799001082656985
Call fail rate in 100 simulations = 3.841999999999999% (mean of 38.42 fails in 1000 calls)
